## Source audio from local or internet

In [9]:
from pytube import YouTube

video_url = "https://www.youtube.com/watch?v=gbjMZq56iws&t=81s"
YouTube(video_url).streams.filter(only_audio=True).first().download(filename="output/output.mp3")


'/home/mohcine/work/personal/whisper/output/output.mp3'

In [3]:
from langchain import OpenAI
def basic_question(prompt):
  llm = OpenAI(
    openai_api_key = "anyValueYouLike",
    temperature = 0,
    openai_api_base = "http://172.19.208.1:1300/v1",
    max_tokens = 30,
  )
  response = llm(prompt=prompt)
  print(response)
  return response

basic_question("What is an AI?")
#basic_question("say this is a test")


An AI, or artificial intelligence, refers to a computer system that can perform tasks that typically require human intelligence, such as visual perception, speech


'\nAn AI, or artificial intelligence, refers to a computer system that can perform tasks that typically require human intelligence, such as visual perception, speech'

## Audio to text

#### whisper v3

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(task="automatic-speech-recognition",
                model="openai/whisper-large-v3",
                torch_dtype=torch.
                float16,
                device="cuda:0")

In [ ]:
result_text = pipe("output.mp3")["text"]


#### distil-whisper

In [13]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "distil-whisper/distil-large-v2"

model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)

model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=256,
    chunk_length_s=15,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)


In [14]:
result_text = pipe("output/output.mp3")["text"]
with open('output/output.txt', 'w') as file:
    # Your string to write to the file
    my_string = result_text
    
    # Write the string to the file
    file.write(my_string)

result_text

" In a previous video, I showed you a number of demos where people were using DPT4WVision and the Open AI Text to Speech API to create video narrators. So I'll walk you through a step-by-step process of how exactly to do that inside a Google CoLab notebook. Everything you're about to see is based on this bookbook from OpenE Act. I'm going to put a link to this in the description. Okay, so here is how the workflow is going to look like. We will provide an input video. Now GPD4 does not understand videos by itself. We will have to convert those into frames and then feed those frames into GPT4 with vision. Then in GPT4 with vision will generate a description of all the frames so whatever it sees in the frames. Then we will use the newly released text-to-speech API from OpenAI to convert that description into speech. And then we will need to combine both the original input video as well as the generated audio to create a new video. Now I'll walk you through a step-by-step process how to do

## Text to speech

#### gtts

In [4]:
from gtts import gTTS
import os

language = "fr"
#language = "en"

text = "Hello Mohcine, how are you today?"
tts = gTTS(text, lang='en', tld="us")
tts.save("speech.mp3")

# Play the converted file
os.system("mpg321 speech.mp3")


High Performance MPEG 1.0/2.0/2.5 Audio Player for Layer 1, 2, and 3.
Version 0.3.2-1 (2012/03/25). Written and copyrights by Joe Drew,
now maintained by Nanakos Chrysostomos and others.
Uses code from various people. See 'README' for more!
THIS SOFTWARE COMES WITH ABSOLUTELY NO WARRANTY! USE AT YOUR OWN RISK!
tcgetattr(): Inappropriate ioctl for device

Playing MPEG stream from speech.mp3 ...
MPEG 2.0 layer III, 32 kbit/s, 24000 Hz mono
                                                                            
[0:02] Decoding of speech.mp3 finished.


0

#### TTS

In [23]:
from TTS.api import TTS

device = "cuda:0" if torch.cuda.is_available() else "cpu"

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2")
tts.to(device=device)
# generate speech by cloning a voice using default settings
tts.tts_to_file(text="It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
                file_path="output/output.mp3",
                speaker_wav="output/output.mp3",
                language="en")




 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
["It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent."]
 > Processing time: 18.099616050720215
 > Real-time factor: 3.0385593092823484


'output/output.mp3'

In [2]:
from transformers import pipeline
import scipy

synthesiser = pipeline("text-to-speech", "suno/bark-small")

speech = synthesiser("Hello, my dog is cooler than you!", forward_params={"do_sample": True})

scipy.io.wavfile.write("bark_out.wav", rate=speech["sampling_rate"], data=speech["audio"])


(…)suno/bark-small/resolve/main/config.json:   0%|          | 0.00/8.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

(…)mall/resolve/main/generation_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

(…)small/resolve/main/tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

(…)o/suno/bark-small/resolve/main/vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

(…)o/bark-small/resolve/main/tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

(…)all/resolve/main/special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


KeyboardInterrupt: 

In [6]:
import subprocess
import os

def record_audio(duration=10, filename="output/recording"):
    wav_filename = f"{filename}.wav"
    mp3_filename = f"{filename}.mp3"

    # Record audio in WAV format
    arecord_command = ["arecord", "-d", str(duration), "-f", "cd", "-t", "wav", wav_filename]
    subprocess.run(arecord_command)

    # Convert WAV to MP3
    lame_command = ["lame", "-b", "192", wav_filename, mp3_filename]
    subprocess.run(lame_command)

    # Optionally, remove the WAV file after conversion
    os.remove(wav_filename)

    return mp3_filename

mp3_file = record_audio()


Recording WAVE 'output/recording.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo
LAME 3.100 64bits (http://lame.sf.net)
Using polyphase lowpass filter, transition band: 18671 Hz - 19205 Hz
Encoding output/recording.wav to output/recording.mp3
Encoding as 44.1 kHz j-stereo MPEG-1 Layer III (7.3x) 192 kbps qval=3
    Frame          |  CPU time/estim | REAL time/estim | play/CPU |    ETA 
     0/       ( 0%)|    0:00/     :  |    0:00/     :  |         x|     :  
00:10--------------------------------------------------------------------------
   kbps      %     %
     0/384    ( 0%)|    0:00/    0:00|    0:00/    0:00|   0.0000x|    0:00 
00:10--------------------------------------------------------------------------
   kbps      %     %
   100/384    (26%)|    0:00/    0:00|    0:00/    0:00|   165.98x|    0:00 
--------------------00:07------------------------------------------------------
   kbps        MS  %     long  %
   200/384    (52%)|    0:00/    0:00|    0:00/    0:00|

In [14]:
import sounddevice as sd
import soundfile as sf
import numpy as np

def record_audio(duration=3, fs=44100, filename="output/recording.wav"):
    """
    Record audio from the microphone and save it to a file.

    Args:
    duration (int): Duration of the recording in seconds.
    fs (int): Sampling rate (samples per second).
    filename (str): Path to save the audio file.
    """
    print("Recording...")
    audio_data = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()  # Wait until recording is finished
    sf.write(filename, audio_data, fs)
    print(f"Recording finished. File saved as {filename}")

# Example usage
record_audio()


Recording...
Recording finished. File saved as output/recording.wav


In [15]:
import sounddevice as sd
import soundfile as sf
import numpy as np

def record_audio(duration=10, fs=44100, filename="output/recording.wav"):
    try:
        print("Recording...")
        audio_data = sd.rec(int(duration * fs), samplerate=fs, channels=2)
        sd.wait()  # Wait until recording is finished
        print("Recording finished, saving file...")
        sf.write(filename, audio_data, fs)
        print(f"File saved as {filename}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
record_audio()


Recording...
Recording finished, saving file...
File saved as output/recording.wav


In [6]:
from ctransformers import AutoModelForCausalLM, AutoTokenizer


model = AutoModelForCausalLM.from_pretrained("TheBloke/dolphin-2.2.1-mistral-7B-GGUF",
                                            model_file="dolphin-2.2.1-mistral-7b.Q4_K_M.gguf",
                                            model_type="mistral",
                                            gpu_layers=20,
                                            max_new_tokens=50,
                                            mlock=True,
                                            threads=8,
                                            temperature=0.2)



Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
model.config


Config(top_k=40, top_p=0.95, temperature=0.8, repetition_penalty=1.1, last_n_tokens=64, seed=-1, batch_size=8, threads=-1, max_new_tokens=50, stop=None, stream=False, reset=True, context_length=-1, gpu_layers=20, mmap=True, mlock=False)